### 🌿 STEP 1-1 - Multi-turn 1. ConversationBufferWindowMemory

In [ ]:
import os
import torch
import dotenv
from operator import itemgetter
from slack_bolt import App
from slack_bolt.adapter.socket_mode import SocketModeHandler
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI
from langchain_teddynote import logging
from langchain_huggingface import HuggingFacePipeline
from langchain.memory import ConversationBufferMemory
from langchain.callbacks.base import BaseCallbackHandler
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.chat_models import ChatOllama
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

dotenv.load_dotenv()
logging.langsmith("RAG-Slack-Bot")
app = App(token=os.environ.get("SLACK_BOT_TOKEN"))
llm = ChatOllama(model="EEVE-Korean-10.8B:latest", max_tokens=100, temperature=0)

In [ ]:
template = """ You are an assistant who can help with a variety of tasks.  
Your name is KPT. Please be sure to answer in Korean. Can provide valuable insight and information on a wide range of topics. 
They can also help you with specific questions and chat about specific topics. 
Please refer to the "Chat History" and keep your answer short and concise with 2-3 sentences.
If the answer is incorrect, please mention "unsure answer".

#Chat History: {history}

#Human: {human_input}

#Assistant: 

"""

prompt = PromptTemplate(input_variables=["history", "human_input"], template=template)


# Function to create a new LLMChain with a fresh memory
def create_chatgpt_chain():
    memory = ConversationBufferWindowMemory(k=3)
    return LLMChain(llm=llm, prompt=prompt, verbose=True, memory=memory)


# Create a new LLMChain instance with fresh memory
chatgpt_chain = create_chatgpt_chain()

In [ ]:
# Event handler for Slack
@app.event("app_mention")
def handle_app_mention_events(body, say, logger):
    message = body["event"]["text"]

    output = chatgpt_chain.predict(human_input=message)
    say(output)


# Message handler for Slack
@app.message(".*")
def message_handler(message, say, logger):
    print(message)

    output = chatgpt_chain.predict(human_input=message["text"])
    say(output)


# Start your app
if __name__ == "__main__":
    SocketModeHandler(app, os.environ["SLACK_APP_TOKEN"]).start()

### 🌿 STEP 2 - Multi-turn 2. RunnableWithMessageHistory

`RunnableWithMessageHistory` 의 초기값에 `session_id` 키를 Default 로 삽입하는 것을 볼 수 있으며,  

이 코드로 인하여 `RunnableWithMessageHistory` 는 대화 스레드 관리를 `session_id` 로 한다는 것을 간접적으로 알 수 있습니다.   

즉, 대화 스레드별 관리는 `session_id` 별로 구현함을 알 수 있습니다.

같은 `session_id` 를 입력하면 이전 대화 스레드의 내용을 가져오기 때문에 이어서 대화가 가능합니다!

   

```python
if history_factory_config:
    _config_specs = history_factory_config
else:
    # If not provided, then we'll use the default session_id field
    _config_specs = [
        ConfigurableFieldSpec(
            id="session_id",
            annotation=str,
            name="Session ID",
            description="Unique identifier for a session.",
            default="",
            is_shared=True,
        ),
    ]
```

따라서, `invoke()` 시 `config={"configurable": {"session_id": "세션ID입력"}}` 코드를 반드시 지정해 주어야 합니다.

In [2]:
import os
import torch
import dotenv
from operator import itemgetter
from slack_bolt import App
from slack_bolt.adapter.socket_mode import SocketModeHandler
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI
from langchain_teddynote import logging
from langchain_huggingface import HuggingFacePipeline
from langchain.schema import HumanMessage, AIMessage
from langchain.memory import ConversationBufferMemory
from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.chat_message_histories import ChatMessageHistory

dotenv.load_dotenv()
logging.langsmith("RAG-Slack-Bot")
app = App(token=os.environ.get("SLACK_BOT_TOKEN"))
llm = ChatOllama(model="EEVE-Korean-10.8B:latest", max_tokens=100, temperature=0)

LangSmith 추적을 시작합니다.
[프로젝트명]
RAG-Slack-Bot


In [3]:
template = """ You are an assistant who can help with a variety of tasks.  
Your name is KPT. Please be sure to answer in Korean. Can provide valuable insight and information on a wide range of topics. 
They can also help you with specific questions and chat about specific topics. 
Please refer to the "Chat History" and keep your answer short and concise with 2-3 sentences.
If the answer is incorrect, please mention "unsure answer".

#Chat History: {history}

#Human: {human_input}

#Assistant: 

"""

prompt = PromptTemplate(input_variables=["history", "human_input"], template=template)

chatgpt_chain = LLMChain(
    llm=llm, prompt=prompt, output_parser=StrOutputParser(), verbose=True
)

/opt/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


### Kernel 실행

In [4]:
# store = {}  # 세션 기록을 저장할 딕셔너리


# def get_session_history(session_id):
#     if session_id not in store:
#         store[session_id] = ChatMessageHistory()
#     return store[session_id]


# def format_history(history):
#     messages = []
#     for msg in history.messages:
#         if isinstance(msg, HumanMessage):
#             messages.append(f"Human: {msg.content}")
#         elif isinstance(msg, AIMessage):
#             messages.append(f"AI: {msg.content}")
#     return "\n".join(messages)


# chain_with_history = RunnableWithMessageHistory(
#     chatgpt_chain,
#     get_session_history,  # 세션 기록을 가져오는 함수
#     input_messages_key="human_input",  # 사용자의 질문이 템플릿 변수에 들어갈 key
#     history_messages_key="history",  # 기록 메시지의 키
# )


# def main():
#     session_id = "default"  # 고정된 세션 ID를 사용

#     while True:
#         human_input = input("입력: ")  # 사용자 입력을 콘솔에서 받기
#         if human_input.lower() in ["exit", "quit"]:
#             break  # 종료 조건

#         history = get_session_history(session_id)
#         formatted_history = format_history(history)

#         response = chain_with_history.invoke(
#             {"human_input": human_input, "history": formatted_history},
#             {"configurable": {"session_id": session_id}},  # config에 session_id 추가
#         )

#         response_text = response.get("text", "응답을 생성하는 데 문제가 발생했습니다.")
#         print(f"AI: {response_text}")

#         # 세션 기록 업데이트
#         history.add_message(HumanMessage(human_input))
#         history.add_message(AIMessage(response_text))


# if __name__ == "__main__":
#     main()

### Slack 실행

In [ ]:
store = {}  # 세션 기록을 저장할 딕셔너리


def get_session_history(
    session_id,
):  # 주어진 세션 ID에 대한 대화 기록을 가져옵니다. 세션 ID가 처음 요청된 경우에는 새로운 대화 기록 객체를 생성하여 저장합니다.
    print(f"[Conversation Session ID]: {session_id}")
    if session_id not in store:  # 세션 ID가 store에 없는 경우
        store[session_id] = (
            ChatMessageHistory()
        )  # 새로운 ChatMessageHistory 객체를 생성하여 store에 저장
    return store[session_id]  # 해당 세션 ID에 대한 세션 기록 반환


def format_history(history):  # 대화 기록 객체를 포맷하여 문자열로 변환합니다.
    messages = []
    for msg in history.messages:
        if isinstance(msg, HumanMessage):  # 메시지가 HumanMessage 타입인 경우
            messages.append(
                f"Human: {msg.content}"
            )  # 'Human: ' 접두어와 함께 메시지 내용 추가
        elif isinstance(msg, AIMessage):  # 메시지가 AIMessage 타입인 경우
            messages.append(
                f"AI: {msg.content}"
            )  # 'AI: ' 접두어와 함께 메시지 내용 추가
    return "\n".join(messages)  # 메시지 리스트를 줄 바꿈으로 결합하여 문자열 반환


chain_with_history = RunnableWithMessageHistory(
    chatgpt_chain,
    get_session_history,  # 세션 기록을 가져오는 함수
    input_messages_key="human_input",  # 사용자의 질문이 템플릿 변수에 들어갈 key
    history_messages_key="history",  # 기록 메시지의 키
)


# Slack 이벤트 핸들러
@app.event("app_mention")
def handle_app_mention_events(body, say, logger):
    session_id = body["event"]["channel"]  # 이벤트가 발생한 채널 ID를 세션 ID로 사용
    message = body["event"]["text"]  # 메시지 내용을 가져옵니다.
    history = get_session_history(session_id)  # 해당 세션 ID의 대화 기록을 가져옵니다.
    formatted_history = format_history(history)  # 대화 기록을 포맷하여 문자열로 변환

    # chain_with_history를 사용하여 응답 생성
    response = chain_with_history.invoke(
        {"human_input": message, "history": formatted_history},
        config={"configurable": {"session_id": session_id}},
    )

    # 응답 딕셔너리에서 'text' 필드만 추출하여 슬랙에 전송합니다.
    response_text = response.get("text", "응답을 생성하는 데 문제가 발생했습니다.")
    say(response_text)


# Slack 메시지 핸들러
@app.message(".*")
def message_handler(message, say, logger):
    session_id = message["channel"]
    text = message["text"]
    history = get_session_history(session_id)
    formatted_history = format_history(history)

    # chain_with_history를 사용하여 응답 생성
    response = chain_with_history.invoke(
        {"human_input": text, "history": formatted_history},
        config={"configurable": {"session_id": session_id}},
    )

    # 응답을 문자열로 변환하여 슬랙에 전송
    response_text = response.get("text", "응답을 생성하는 데 문제가 발생했습니다.")
    say(response_text)


if __name__ == "__main__":
    SocketModeHandler(app, os.environ["SLACK_APP_TOKEN"]).start()

### 🌿 STEP 3 - Retriever

#### 👉🏻 NotionDirectoryLoader
https://logan-vendrix.medium.com/create-your-own-notion-chatbot-with-langchain-openai-and-streamlit-fcb385f432a2

#### 👉🏻 RecursiveCharacterTextSplitter
https://api.python.langchain.com/en/latest/character/langchain_text_splitters.character.RecursiveCharacterTextSplitter.html

In [4]:
import os
import urllib.parse
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.schema import HumanMessage, AIMessage
from langchain_community.vectorstores import FAISS
from langchain_community.chat_models import ChatOllama
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.document_loaders import NotionDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory

In [9]:
def decode_url(encoded_url):
    return urllib.parse.unquote(encoded_url)


# 문서의 URL 인코딩된 경로를 디코딩하는 함수
def decode_documents(docs):
    decoded_docs = []
    for doc in docs:
        decoded_content = decode_url(doc.page_content)
        decoded_doc = doc  # 기존 문서 객체를 사용
        decoded_doc.page_content = decoded_content  # URL 디코딩된 내용으로 업데이트
        decoded_docs.append(decoded_doc)
    return decoded_docs


# 파일 로딩 및 디코딩 작업
def load_and_decode_documents(folder_path):
    # STEP 1. 문서 로드(Load Documents)
    loader = NotionDirectoryLoader(folder_path)
    documents = loader.load()

    # STEP 2: 문서 분할(Split Documents)
    markdown_splitter = RecursiveCharacterTextSplitter(
        separators=["#", "##", "###", "\n\n", "\n", "."],
        chunk_size=1500,
        chunk_overlap=100,
    )
    docs = markdown_splitter.split_documents(documents)

    # URL 인코딩 디코딩
    decoded_docs = decode_documents(docs)

    return decoded_docs


# Notion content 폴더 경로
folder_path = "notion_content"
decoded_documents = load_and_decode_documents(folder_path)

# 디코딩된 문서 내용 출력
print(f"*** Documents length: {len(decoded_documents)}")
print(f"*** Decoded content for document 1: {decoded_documents[0].page_content}")

*** Documents length: 219
*** Decoded content for document 1: # 대학교 이수 과목 별 제출 보고서 및 발표 자료

[[과목별 보고서 & 발표 자료]-어셈블리언어](대학교 이수 과목 별 제출 보고서 및 발표 자료 9baa031ab64a474e8222d328d8b554f3/[과목별 보고서 & 발표 자료]-어셈블리언어 c05274e93077416ba857cf01009bbd83.csv)


In [12]:
# decoded_documents[1:10]

In [10]:
# STEP 3: 임베딩(Embedding) 생성
embeddings = OpenAIEmbeddings()
# STEP 4: DB 생성(Create DB) 및 저장
vectorstore = FAISS.from_documents(documents=decoded_documents, embedding=embeddings)
# STEP 5 : 검색기(Retriever) 생성
retriever = vectorstore.as_retriever()

In [4]:
retriever.invoke("정보보호개론 성적은?")

[Document(metadata={'source': 'notion_content/🏫 대학교 79bb72c98b654231a8d07dc2f8f324f1/대학교 이수 과목 및 성적 094fc5a517e44a0ea05c6b81d867901d/[과목 성적] (1) dbd0203c06854a9e85e99c00194fb1e5/정보보호개론-[전공] 195677ad9ea444108cda5dc447384488.md'}, page_content='# 정보보호개론-[전공]\n\nTags: 1학년, 2학기\nGrade: A+'),
 Document(metadata={'source': 'notion_content/🏫 대학교 79bb72c98b654231a8d07dc2f8f324f1/대학교 이수 과목 및 성적 094fc5a517e44a0ea05c6b81d867901d/[과목 성적] (1) dbd0203c06854a9e85e99c00194fb1e5/정보이론-[전공] 00fcccf2f4d541758925690b919730ba.md'}, page_content='# 정보이론-[전공]\n\nTags: 2학기, 3학년\nGrade: A0\n\n##'),
 Document(metadata={'source': 'notion_content/🏫 대학교 79bb72c98b654231a8d07dc2f8f324f1/대학교 이수 과목 및 성적 094fc5a517e44a0ea05c6b81d867901d.md'}, page_content='# 대학교 이수 과목 및 성적\n\n[[과목 성적] (1)](대학교 이수 과목 및 성적 094fc5a517e44a0ea05c6b81d867901d/[과목 성적] (1) dbd0203c06854a9e85e99c00194fb1e5.csv)'),
 Document(metadata={'source': 'notion_content/🏫 대학교 79bb72c98b654231a8d07dc2f8f324f1/대학교 이수 과목 및 성적 094fc5a517

### Retriever 테스트 코드
* 간단한 chain 구현

In [12]:
from langchain_core.runnables import RunnablePassthrough


prompt = PromptTemplate.from_template(
    """You are an assistant who can help with a variety of tasks.  
Your name is KPT. Please be sure to answer in Korean. Can provide valuable insight and information on a wide range of topics. 
They can also help you with specific questions and chat about specific topics. 
Please refer to the "Chat History" and keep your answer short and concise with 2-3 sentences.
If the answer is incorrect, please mention "unsure answer".

#Question: 
{question} 

#Context: 
{context} 

#Answer:"""
)

# llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)
llm = ChatOllama(model="EEVE-Korean-10.8B:latest", max_tokens=100, temperature=0)

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# question = "대학교 교내 활동을 무엇을 했을까? 관련 문서로 된 csv나 pdf 찾아줘."
question = "취약점분석기술 과목 보고서를 작성한 적이 있었나? 있다면 관련 문서로 된 csv 형태의 파일이나 pdf 형태의 파일을 찾아주고 파일이 있는 경로도 알려줘."
response = chain.invoke(question)
print(response)

취약점분석기술 과목 보고서와 관련된 파일들을 찾았습니다. 다음은 관련 문서들입니다:

* 취약점분석기술_4조_최종보고서.pdf (취약점분석기술_4조/취약점분석기술_4조_최종보고서.pdf)
* 취약점분석기술_4조_중간보고서.pdf (취약점분석기술_4조/취약점분석기술_4조_중간보고서.pdf)
* 20194577_김수현_취약점분석기술 개인보고서.pdf (20194577_김수현_취약점분석기술 개인보고서/20194577_김수현_취약점분석기술_개인보고서.pdf)

해당 파일들은 대학교 이수 과목 별 제출 보고서 및 발표 자료 폴더에 위치해 있습니다 (대학교 이수 과목 별 제출 보고서 및 발표 자료 9baa031ab64a474e8222d328d8b554f3).


### Retriever 테스트 코드
* LLMChain으로 구현
* 코드 실행 시 다음과 같은 프롬프트는 환각 현상이 보여 수정 진행하였음

==============================================================================================================================

[before]  

```python   
"""You are an assistant who can help with a variety of tasks.  
Your name is KPT. Please be sure to answer in Korean. Can provide valuable insight and information on a wide range of topics. 
They can also help you with specific questions and chat about specific topics. 
Please refer to the "Chat History" and keep your answer short and concise with 2-3 sentences.
If the answer is incorrect, please mention "unsure answer"."""
```

취약점 분석 기술 과목 보고서 작성에 대한 문의에 도움을 드리겠습니다. 제공해주신 맥락과 태그를 바탕으로, 다음과 같은 관련 문서들을 찾았습니다:

1. 취약점분석기술_4조_최종보고서.pdf (PDF 파일)
2. 취약점분석기술_4조_중간보고서.pdf (PDF 파일)
3. 20194577_김수현_취약점분석기술 개인보고서.pdf (PDF 파일)

이 문서들은 모두 취약점 분석 기술 과목과 관련이 있으며, 보고서 형식으로 작성되었습니다. 해당 파일들은 제공된 URL에서 찾아보실 수 있습니다:

- 취약점분석기술_4조_최종보고서.pdf: <https://drive.google.com/file/d/020d2d377f7142f389ff265e7df...>
- 취약점분석기술_4조_중간보고서.pdf: <https://drive.google.com/file/d/bf99519803114166ac6ba...>
- 20194577_김수현_취약점분석기술 개인보고서.pdf: <https://drive.google.com/file/d/fa8871ee...>

이 파일들이 도움이 되길 바랍니다! 추가적인 질문이 있거나 더 자세한 정보가 필요하시면 언제든지 문의해 주세요.

==============================================================================================================================

[after]

```python    
"""Your name is KPT. Please be sure to answer in Korean. Can provide valuable insight and information on a wide range of topics. 
They can also help you with specific questions and chat about specific topics. 
Please refer to the "Chat History" and keep your answer short and concise with 2-3 sentences.
If the answer is incorrect, please mention "unsure answer".
*** Be sure to answer only using the information given, and be sure to include a separator such as '/' in the file path and specify folder information as well. ***""" 
```

취약점분석기술 과목 보고서 작성을 위한 관련 문서로 된 CSV 형태의 파일이나 PDF 형태의 파일을 찾고자 한다면, 제공된 맥락에 있는 파일들을 참고하세요. 다음은 해당 파일들의 경로와 이름입니다:

1. 취약점분석기술_4조_최종보고서.pdf (취약점분석기술 4조 최종 보고서)
2. 취약점분석기술_4조_중간보고서.pdf (취약점분석기술 4조 중간 보고서)
3. 20194577_김수현_취약점분석기술 개인보고서.pdf (20194577 김수현 취약점분석기술 개인 보고서)

이 파일들은 제공된 맥락에 있는 폴더 안에 위치해 있습니다:

- 취약점분석기술_4조_최종보고서.pdf: 취약점분석기술_4조/취약점분석기술_4조_최종보고서.pdf
- 취약점분석기술_4조_중간보고서.pdf: 취약점분석기술_4조/취약점분석기술_4조_중간보고서.pdf
- 20194577_김수현_취약점분석기술 개인보고서.pdf: 20194577_김수현/20194577_김수현_취약점분석기술_개인보고서.pdf

이 파일들을 찾으려면 제공된 경로와 이름을 참고하세요.

In [15]:
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# retriever 설정
retriever = vectorstore.as_retriever()

# 프롬프트 템플릿 설정
prompt = PromptTemplate.from_template(
    """Your name is KPT. Please be sure to answer in Korean. Can provide valuable insight and information on a wide range of topics. 
They can also help you with specific questions and chat about specific topics. 
Please refer to the "Chat History" and keep your answer short and concise with 2-3 sentences.
If the answer is incorrect, please mention "unsure answer".
*** Be sure to answer only using the information given, and be sure to include a separator such as '/' in the file path and specify folder information as well. ***.

#Question: 
{question} 

#Context: 
{context} 

#Answer:"""
)

# LLM 설정
llm = ChatOllama(model="EEVE-Korean-10.8B:latest", max_tokens=100, temperature=0)

# LLMChain 설정
chain = LLMChain(prompt=prompt, llm=llm, output_parser=StrOutputParser(), verbose=True)


def retrieve_documents(human_input, num_results=5):
    # retriever를 이용하여 문서 검색
    results = retriever.get_relevant_documents(human_input, k=num_results)
    # 검색된 결과를 하나의 문자열로 병합
    return "\n".join([doc.page_content for doc in results])


# 질문과 문서 검색 결과를 체인에 넣어 실행
def main():
    question = "취약점분석기술 과목 보고서를 작성한 적이 있었나? 있다면 관련 문서로 된 csv 형태의 파일이나 pdf 형태의 파일을 찾아주고 파일이 있는 경로도 알려줘."

    context = retrieve_documents(question, num_results=5)
    inputs = {"question": question, "context": context}

    response = chain.invoke(inputs)
    response_text = response.get("text", "응답을 생성하는 데 문제가 발생했습니다.")
    print(response_text)


if __name__ == "__main__":
    main()



> Entering new LLMChain chain...
Prompt after formatting:
Your name is KPT. Please be sure to answer in Korean. Can provide valuable insight and information on a wide range of topics. 
They can also help you with specific questions and chat about specific topics. 
Please refer to the "Chat History" and keep your answer short and concise with 2-3 sentences.
If the answer is incorrect, please mention "unsure answer".
*** Be sure to answer only using the information given, and be sure to include a separator such as '/' in the file path and specify folder information as well. ***.

#Question: 
취약점분석기술 과목 보고서를 작성한 적이 있었나? 있다면 관련 문서로 된 csv 형태의 파일이나 pdf 형태의 파일을 찾아주고 파일이 있는 경로도 알려줘. 

#Context: 
[[과목별 보고서 & 발표 자료]-취약점분석기술 (Team Project)](대학교 이수 과목 별 제출 보고서 및 발표 자료 9baa031ab64a474e8222d328d8b554f3/[과목별 보고서 & 발표 자료]-취약점분석기술 b837089ae9044ffcbb7911edef07eaa7.csv)
# 취약점분석기술_4조_최종보고서

Tags: Report

[취약점분석기술_4조_최종보고서.pdf](취약점분석기술_4조_최조

### Retriever & Model 연동 - Kernel 실행

In [22]:
llm = ChatOllama(model="EEVE-Korean-10.8B:latest", max_tokens=100, temperature=0)

template = """Your name is KPT. Please be sure to answer in Korean. 
Can provide valuable insight and information on a wide range of topics. 
They can also help you with specific questions and chat about specific topics. 
Please refer to the "Chat History" and keep your answer short and concise with 2-3 sentences.
If the answer is incorrect, please mention "unsure answer".
*** Please mention and answer only the content related to Human's question! If the purpose is to greet you, just say hello, and if you are asking for information, ask them to provide the information. ***
*** Be sure to answer only using the information given, and be sure to include a separator such as '/' in the file path and specify folder information as well. ***

#Chat History: {history}

#Context: {context}

#Human: {human_input}

#Assistant: 

"""

prompt = PromptTemplate(
    input_variables=["history", "human_input", "context"], template=template
)

chatgpt_chain = LLMChain(
    llm=llm, prompt=prompt, output_parser=StrOutputParser(), verbose=True
)

In [23]:
# 세션 기록을 저장할 딕셔너리
store = {}


def get_session_history(session_id):
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


def format_history(history):
    messages = []
    for msg in history.messages:
        if isinstance(msg, HumanMessage):
            messages.append(f"Human: {msg.content}")
        elif isinstance(msg, AIMessage):
            messages.append(f"AI: {msg.content}")
    return "\n".join(messages)


def retrieve_documents(human_input):
    # def retrieve_documents(human_input, num_results=5):
    # retriever를 이용하여 문서 검색
    results = retriever.get_relevant_documents(human_input)
    # results = retriever.get_relevant_documents(human_input, k=num_results)
    # 검색된 결과를 하나의 문자열로 병합
    return "\n".join([doc.page_content for doc in results])


chain_with_history = RunnableWithMessageHistory(
    chatgpt_chain,
    get_session_history,  # 세션 기록을 가져오는 함수
    input_messages_key="human_input",  # 사용자의 질문이 템플릿 변수에 들어갈 key
    history_messages_key="history",  # 기록 메시지의 키
)


def main():
    session_id = "default"  # 고정된 세션 ID를 사용

    while True:
        human_input = input("입력: ")  # 사용자 입력을 콘솔에서 받기
        if human_input.lower() in ["exit", "quit"]:
            break  # 종료 조건

        history = get_session_history(session_id)
        formatted_history = format_history(history)

        # 사용자 입력을 기반으로 문서 검색
        context = retrieve_documents(human_input)

        response = chain_with_history.invoke(
            {
                "human_input": human_input,
                "history": formatted_history,
                "context": context,
            },
            {"configurable": {"session_id": session_id}},  # config에 session_id 추가
        )

        response_text = response.get("text", "응답을 생성하는 데 문제가 발생했습니다.")
        print(f"AI: {response_text}")

        # 세션 기록 업데이트
        history.add_message(HumanMessage(human_input))
        history.add_message(AIMessage(response_text))


if __name__ == "__main__":
    main()



> Entering new LLMChain chain...
Prompt after formatting:
Your name is KPT. Please be sure to answer in Korean. 
Can provide valuable insight and information on a wide range of topics. 
They can also help you with specific questions and chat about specific topics. 
Please refer to the "Chat History" and keep your answer short and concise with 2-3 sentences.
If the answer is incorrect, please mention "unsure answer".
*** Please mention and answer only the content related to Human's question! If the purpose is to greet you, just say hello, and if you are asking for information, ask them to provide the information. ***
*** Be sure to answer only using the information given, and be sure to include a separator such as '/' in the file path and specify folder information as well. ***

#Chat History: []

#Context: # 제목 없음
# 어셈블리언어-[전공]

Tags: 1학기, 2학년
Grade: A+
# 전공영어-[전공]

Tags: 1학기, 3학년
Grade: A0
# 환경과에너지-[교양]

Tags: 1학기, 3학년
Grade: A0

#Human: 안녕?

#Assistant: 



> Finished chain.
AI: 안녕

### 🌿 FIANL STEP  - Slack과 Retriever- Multiturn Model 연동

In [5]:
import os
import urllib.parse
from slack_bolt import App
from slack_bolt.adapter.socket_mode import SocketModeHandler
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.schema import HumanMessage, AIMessage
from langchain_community.vectorstores import FAISS
from langchain_community.chat_models import ChatOllama
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.document_loaders import NotionDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory

app = App(token=os.environ["SLACK_BOT_TOKEN"])

In [7]:
# 프롬프트 템플릿 설정
template = """Your name is KPT. Please be sure to answer in Korean. 
Can provide valuable insight and information on a wide range of topics. 
They can also help you with specific questions and chat about specific topics. 
Please refer to the "Chat History" and keep your answer short and concise with 2-3 sentences.
If the answer is incorrect, please mention "unsure answer".
*** Please mention and answer only the content related to Human's question! If the purpose is to greet you, just say hello, and if you are asking for information, ask them to provide the information. ***
*** Be sure to answer only using the information given, and be sure to include a separator such as '/' in the file path and specify folder information as well. ***

#Chat History: {history}

#Context: {context}

#Human: {human_input}

#Assistant: 

"""

prompt = PromptTemplate(
    input_variables=["history", "human_input", "context"], template=template
)

# LLM 설정
llm = ChatOllama(model="EEVE-Korean-10.8B:latest", max_tokens=100, temperature=0)

# LLMChain 설정
chatgpt_chain = LLMChain(
    llm=llm, prompt=prompt, output_parser=StrOutputParser(), verbose=True
)

In [ ]:
# 세션 기록을 저장할 딕셔너리
store = {}


def get_session_history(session_id):
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


def format_history(history):
    messages = []
    for msg in history.messages:
        if isinstance(msg, HumanMessage):
            messages.append(f"Human: {msg.content}")
        elif isinstance(msg, AIMessage):
            messages.append(f"AI: {msg.content}")
    return "\n".join(messages)


def retrieve_documents(human_input, num_results=5):
    # retriever를 이용하여 문서 검색
    results = retriever.get_relevant_documents(human_input, k=num_results)
    # 검색된 결과를 하나의 문자열로 병합
    return "\n".join([doc.page_content for doc in results])


# chain_with_history 설정
chain_with_history = RunnableWithMessageHistory(
    chatgpt_chain,
    get_session_history,  # 세션 기록을 가져오는 함수
    input_messages_key="human_input",  # 사용자의 질문이 템플릿 변수에 들어갈 key
    history_messages_key="history",  # 기록 메시지의 키
)


# Slack 이벤트 핸들러
@app.event("app_mention")
def handle_app_mention_events(body, say, logger):
    session_id = body["event"]["channel"]  # 이벤트가 발생한 채널 ID를 세션 ID로 사용
    message = body["event"]["text"]  # 메시지 내용을 가져옵니다.
    history = get_session_history(session_id)  # 해당 세션 ID의 대화 기록을 가져옵니다.
    formatted_history = format_history(history)  # 대화 기록을 포맷하여 문자열로 변환

    # 컨텍스트를 검색하여 문서 병합
    context = retrieve_documents(message)

    # chain_with_history를 사용하여 응답 생성
    response = chain_with_history.invoke(
        {"human_input": message, "history": formatted_history, "context": context},
        {"configurable": {"session_id": session_id}},  # config에 session_id 추가
    )

    # 응답 딕셔너리에서 'text' 필드만 추출하여 슬랙에 전송합니다.
    response_text = response.get("text", "응답을 생성하는 데 문제가 발생했습니다.")
    say(response_text)

    # 세션 기록 업데이트
    history.add_message(HumanMessage(message))
    history.add_message(AIMessage(response_text))


# Slack 메시지 핸들러
@app.message(".*")
def message_handler(message, say, logger):
    session_id = message["channel"]
    text = message["text"]
    history = get_session_history(session_id)
    formatted_history = format_history(history)

    # 컨텍스트를 검색하여 문서 병합
    context = retrieve_documents(text)

    # chain_with_history를 사용하여 응답 생성
    response = chain_with_history.invoke(
        {"human_input": text, "history": formatted_history, "context": context},
        {"configurable": {"session_id": session_id}},  # config에 session_id 추가
    )

    # 응답을 문자열로 변환하여 슬랙에 전송
    response_text = response.get("text", "응답을 생성하는 데 문제가 발생했습니다.")
    say(response_text)

    # 세션 기록 업데이트
    history.add_message(HumanMessage(text))
    history.add_message(AIMessage(response_text))


if __name__ == "__main__":
    SocketModeHandler(app, os.environ["SLACK_APP_TOKEN"]).start()